In [64]:
import uproot
import awkward as ak
import numpy as np
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema,DelphesSchema
from coffea import hist

In [46]:
events_11 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/bbDM_2HDMa_MH3_1500_MH4_150_Mchi_10_HLLHC_GEN_v3/delphes_ntuple_one.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

events_12 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/bbDM_2HDMa_MH3_1500_MH4_150_Mchi_10_HLLHC_GEN_v3/delphes_ntuple_two.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()


events_21 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/bbDM_2HDMa_MH3_1500_MH4_250_Mchi_10_HLLHC_GEN_v3/delphes_ntuple_one.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

events_22 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/bbDM_2HDMa_MH3_1500_MH4_250_Mchi_10_HLLHC_GEN_v3/delphes_ntuple_two.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()   

events_31 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/bbDM_2HDMa_MH3_1500_MH4_300_Mchi_10_HLLHC_GEN_v3/delphes_ntuple_one.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

events_32 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/bbDM_2HDMa_MH3_1500_MH4_300_Mchi_10_HLLHC_GEN_v3/delphes_ntuple_two.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()   

events_41 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/bbDM_2HDMa_MH3_1500_MH4_400_Mchi_10_HLLHC_GEN_v3/delphes_ntuple_one.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

events_42 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/bbDM_2HDMa_MH3_1500_MH4_400_Mchi_10_HLLHC_GEN_v3/delphes_ntuple_two.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()   

/home/users/ewallace/miniconda3/envs/coffeadev/lib/python3.9/site-packages/coffea/nanoevents/mapping/uproot.py:74: UserWarning: Found duplicate branch fatjet_pt in <TTree 'mytree' (103 branches) at 0x7f51565d28b0>, taking first instance
  warnings.warn(
/home/users/ewallace/miniconda3/envs/coffeadev/lib/python3.9/site-packages/coffea/nanoevents/mapping/uproot.py:74: UserWarning: Found duplicate branch fatjet_pt in <TTree 'mytree' (103 branches) at 0x7f51563c0a90>, taking first instance
  warnings.warn(
/home/users/ewallace/miniconda3/envs/coffeadev/lib/python3.9/site-packages/coffea/nanoevents/mapping/uproot.py:74: UserWarning: Found duplicate branch fatjet_pt in <TTree 'mytree' (103 branches) at 0x7f5155324850>, taking first instance
  warnings.warn(
/home/users/ewallace/miniconda3/envs/coffeadev/lib/python3.9/site-packages/coffea/nanoevents/mapping/uproot.py:74: UserWarning: Found duplicate branch fatjet_pt in <TTree 'mytree' (103 branches) at 0x7f52915dff10>, taking first instance
 

In [99]:
puppiMET = [[],[],[],[], np.linspace(0,250,11), r'$Puppi\ MET_{pt}$']
genMET = [[],[],[],[], np.linspace(0,80,9), r'$Gen\ MET_{pt}$']
fatjet_pt = [[],[],[],[], np.linspace(200,450,26), r'$Fatjet\ p_{T}$']
fatjet_eta = [[],[],[],[], np.linspace(-4,4,9), r'$Fatjet\ \eta$']
fatjet_phi = [[],[],[],[], np.linspace(-4,4,9), r'$Fatjet\ \phi$']
fatjet_mass = [[],[],[],[], np.linspace(0,200,21), r'$Fatjet\ mass$']
fatjet_msoftdrop = [[],[],[],[], np.linspace(0,150,16), r'$Fatjet\ sd\ mass$']
fatjet_tau1 = [[],[],[],[], np.linspace(0,0.6,13), r'$Fatjet\ \tau_1$']
fatjet_tau2 = [[],[],[],[], np.linspace(0,0.25,6), r'$Fatjet\ \tau_2$']
fatjet_tau3 = [[],[],[],[], np.linspace(0,0.2,5), r'$Fatjet\ \tau_3$']
fatjet_tau4 = [[],[],[],[], np.linspace(0,0.15,4), r'$Fatjet\ \tau_4$']
varis = {'metpuppi_pt': puppiMET, 'genmet_pt': genMET, 'fatjet_pt': fatjet_pt, 'fatjet_eta': fatjet_eta, 
         'fatjet_phi': fatjet_phi, 'fatjet_mass': fatjet_mass, 'fatjet_msoftdrop': fatjet_msoftdrop,
        'fatjet_tau1': fatjet_tau1, 'fatjet_tau2': fatjet_tau2, 'fatjet_tau3': fatjet_tau3, 'fatjet_tau4': fatjet_tau4}

In [84]:
varis

{'metpuppi_pt': [[],
  [],
  [],
  [],
  array([  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.,
         110., 120., 130., 140., 150., 160., 170., 180., 190., 200., 210.,
         220., 230., 240., 250.]),
  '$Puppi\\ MET_{pt}$'],
 'genmet_pt': [[],
  [],
  [],
  [],
  array([  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.]),
  '$Gen\\ MET_{pt}$'],
 'fatjet_pt': [[],
  [],
  [],
  [],
  array([  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.,
         110., 120., 130., 140., 150., 160., 170., 180., 190., 200., 210.,
         220., 230., 240., 250.]),
  '$Fatjet\\ p_{T}$'],
 'fatjet_eta': [[],
  [],
  [],
  [],
  array([-5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.,  5.]),
  '$Fatjet\\ \\eta$'],
 'fatjet_phi': [[],
  [],
  [],
  [],
  array([-5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.,  5.]),
  '$Fatjet\\ \\phi$'],
 'fatjet_mass': [[],
  [],
  [],
  [],
  array([  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.,
 

In [100]:
for vari in varis:
    varis[vari][0] = ak.flatten(ak.concatenate([events_11[vari],events_12[vari]]))
    varis[vari][1] = ak.flatten(ak.concatenate([events_21[vari],events_22[vari]]))
    varis[vari][2] = ak.flatten(ak.concatenate([events_31[vari],events_32[vari]]))
    varis[vari][3] = ak.flatten(ak.concatenate([events_41[vari],events_42[vari]]))

In [90]:
varis

{'metpuppi_pt': [<Array [240, 30.6, 33.6, ... 13.7, 32.4, 1.97] type='439912 * float32[parameters...'>,
  <Array [22.4, 1.5, 77.4, ... 16.7, 46.8, 41.9] type='419916 * float32[parameters...'>,
  <Array [33.3, 31.6, 30.5, ... 116, 11.1, 13.5] type='444911 * float32[parameters...'>,
  <Array [42.2, 28.4, 36.3, ... 18.2, 128, 8.11] type='434913 * float32[parameters...'>,
  array([  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.,
         110., 120., 130., 140., 150., 160., 170., 180., 190., 200., 210.,
         220., 230., 240., 250.]),
  '$Puppi\\ MET_{pt}$'],
 'genmet_pt': [<Array [1.7, 18.2, 0.00636, ... 12.7, 0.000363] type='439912 * float32[parameter...'>,
  <Array [10.2, 0.03, 13.1, ... 0.00199, 0.00472] type='419916 * float32[parameter...'>,
  <Array [0.493, 25, 0.218, ... 0.00422, 0.0154] type='444911 * float32[parameters...'>,
  <Array [0.493, 25, 0.218, ... 3.98, 1.98] type='434913 * float32[parameters={"__...'>,
  array([  0.,  10.,  20.,  30.,  40.,  50.,  60.,

In [91]:
len(varis['metpuppi_pt'][3])

434913

In [92]:
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)

In [101]:
for vari in varis:
    plt.hist(varis[vari][0], bins=varis[vari][4], histtype = 'step', label = 'MH3_1500_MH4_150_Mchi_10')
    plt.hist(varis[vari][1], bins=varis[vari][4], histtype = 'step', label = 'MH3_1500_MH4_250_Mchi_10')
    plt.hist(varis[vari][2], bins=varis[vari][4], histtype = 'step', label = 'MH3_1500_MH4_300_Mchi_10') 
    plt.hist(varis[vari][3], bins=varis[vari][4], histtype = 'step', label = 'MH3_1500_MH4_400_Mchi_10')
    plt.xlabel(varis[vari][5])
    plt.ylabel(r'Events')
    plt.legend(prop={'size': 13})
    plt.savefig('/home/users/ewallace/public_html/HbbMET/delphes_'+str(vari)+'.png')
    plt.clf()

<Figure size 720x720 with 0 Axes>